<a href="https://colab.research.google.com/github/Inyrkz/Politician-db/blob/master/Politician.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!git clone https://github.com/Inyrkz/Politician-db.git

Cloning into 'Politician-db'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [23]:
import pandas as pd
METADATA = pd.read_csv('/content/Politician-db/Politician DS.csv')
METADATA.head(3)

,S/N,Name,Position,Political Party,Elected,Posts
0,1,Okezie Ikpeazu,Current Governor of Abia State,PDP,2019,NaN
1,2,Ahmadu Umaru Fintiri,Current Governor of Adamawa State,PDP,2019,NaN
2,3,Udom Gabriel Emmanuel,Current Governor of Akwa-Ibom State,PDP,2019,NaN


In [24]:
#Removing NaN values
METADATA = METADATA.fillna('')

#Convert the post column string
METADATA["Elected"] = METADATA["Elected"].astype(int).astype(str)

#Joining the other columns into one 
METADATA["General"] = METADATA["Position"] + " " + METADATA["Political Party"] + " " +  METADATA["Elected"]+ " " +  METADATA["Posts"]
METADATA["General"].head()

0         Current Governor of Abia State PDP 2019 
1      Current Governor of Adamawa State PDP 2019 
2    Current Governor of Akwa-Ibom State PDP 2019 
3     Current Governor of Anambra State APGA 2014 
4          Deputy Governor of Abia State PDP 2019 
Name: General, dtype: object

In [28]:
#Importing Libraries
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer object. Remove all english stopwords
TFIDF = TfidfVectorizer(stop_words = 'english')

#Construct the required TF-IDF matrix by fitting and transforming the data
TFIDF_MATRIX = TFIDF.fit_transform(METADATA["General"])

#Let's see the output shape of the matrix
TFIDF_MATRIX.shape

(98, 76)

In [0]:
#Importing liinear_kernel
from sklearn.metrics.pairwise import linear_kernel

#computing the cosine similarity matrix
COSINE_SIM = linear_kernel(TFIDF_MATRIX, TFIDF_MATRIX)

#Construct a reverse map of the indices and name
indices = pd.Series(METADATA.index, index = METADATA['Name']).drop_duplicates()

In [0]:

def get_recommendations(name, COSINE_SIM = COSINE_SIM):
  '''Function that takes in a politician's name as input and output similar politicians'''

  #Get the index of the name
  idx = indices[name]

  #Get the pairwise similarity score of all names with that name
  sim_scores = list(enumerate(COSINE_SIM[idx]))

  #sort the names based on similarity scores
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

  #Get the scores of the 10 most similar politicians
  sim_scores = sim_scores[1:11]

  #Get the names
  names = [i[0] for i in sim_scores] 

  #Return the top 10 similar politicians
  return METADATA['Name'].iloc[names]

In [37]:
PNAME = input("Search Politician's name: ")
get_recommendations(PNAME)

Search Politician's name: Muhammadu Buhari


84        Olusegun Obasanjo
82        Goodluck Jonathan
83      Umaru Musa Yar'Adua
79         Akinwunmi Ambode
46       Babajide Sanwo-Olu
47     Kadri Obafemi Hamzat
85        Femi Gbajabiamila
14           BabaGana Umara
28    Muhammad Inuwa Yahaya
32          Badaru Abubakar
Name: Name, dtype: object